# Environment Setup

This notebook is designed for Python version **3.11.5** and requires several external packages to run correctly. Below is a list of necessary packages:

- **IPython**: For powerful interactive shells.
- **Pillow** (PIL): For image processing capabilities.
- **ipywidgets**: For interactive HTML widgets.
- **joblib**: For lightweight pipelining.
- **matplotlib**: For creating visualizations.
- **networkx**: For creating and manipulating complex networks.
- **numba**: For JIT compilation.
- **numpy**: For support with large, multi-dimensional arrays and matrices.
- **pandas**: For data manipulation and analysis.
- **requests**: For sending HTTP requests.
- **scipy**: For scientific and technical computing.
- **seaborn**: For drawing attractive statistical graphics.
- **scikit-learn** (sklearn): For machine learning.
- **statsmodels**: For statistical modeling.
- **d3blocks**: For creating chord diagrams.

To install these packages, run the following command in a code cell:

```python
!pip install IPython Pillow ipywidgets joblib matplotlib networkx numba numpy pandas requests scipy seaborn scikit-learn statsmodels d3blocks
```

Ensure that all packages are installed before proceeding to run the analyses in this notebook.

In [ ]:
from abm import *
from read_sbml import create_model, read_interaction_tables

# Uncompartmentalized Model

## Loading the Submaps

In [ ]:
air_model = create_model("Example Maps/AIR")

## Visualize Topological Weightings

In [ ]:
phenotype = air_model.get_node_from_name("prostaglandin synthesis")
weightings = air_model.get_influence_scores(phenotypes = [phenotype])["values"][phenotype]

In [ ]:
air_model.highlight_on_map('Biosynthesis of PIM and SPM from AA.xml', max_width = 5000, highlights = weightings).save("test.jpg", dpi = (900,900))

## Load the Data File

In [ ]:
data = pd.read_csv("Data Files/ColitisFCData_pvalues.txt", index_col = 0)

## Perform Enrichment Analysis

In [ ]:
results = air_model.data_enrichment(data)

In [ ]:
air_model.circle_plot(results, figsize = (10,10))

# Compartmental Boolean Model

In [ ]:
sarcopenia_model = create_model("Example Maps/Sarcopenia", compartment_specific = True)

## Boolean Simulation

In [ ]:
for node in sarcopenia_model.nodes:
    if node.compartment == "":
        node.default_state = False

perturbations = {
    sarcopenia_model.get_node_from_name("alcohol consumption"): [-1],
    sarcopenia_model.get_node_from_name("exercise"): [-1],
    sarcopenia_model.get_node_from_name("food intake"): 5*[1] + 10*[-1],
    sarcopenia_model.get_node_from_name("cirrhosis"): [-1],
    sarcopenia_model.get_node_from_name("lep"): [-1],
    sarcopenia_model.get_node_from_name("intestinal dysfunction"): [-1],
}

In [ ]:
sarcopenia_model.show_boolean_simulation("Liver.xml", max_steps = 500, conditions = perturbations)

## Chord Diagram

In [ ]:
masld_model = create_model("Example Maps/MASLD", compartment_specific = True)

In [ ]:
masld_model.chord_plot(highlighted = ["hepatocyte"])

## Spatial Boolean Simulation

In [ ]:
abm = ABM(r"Example Maps/MASLD", 
          agentmaps = ["Hepatocyte.xml", "Hepatocyte Apoptosis.xml", "Vitamins and Trace Elements.xml", "Bile Acids.xml", "Cholesterol Synthesis and Effects.xml"], 
          outsidemaps = ["Digestion and Absorption.xml", "Adipose Tissue.xml", "Endocrine Pancreas.xml"], 
          image = r"Example Maps/MASLD/liver_agents.png",  
          grid_size = 50, 
          agents_per_quadrant = 5, 
          seed = 9157)

In [ ]:
abm.run_abm(steps = 400,food_prob = 100*[1]+100*[-1],
            perturbednodes = ["cirrhosis", "intestinal dysfunction", "alcohol consumption", "portal hypertension", "Exercise", "acot1", "ADM", "Alcohol Consumption"])

In [ ]:
abm.show_activity_plot([
    abm.agent_model.get_node_from_name("tag", compartment = "hepatocyte"),
    abm.agent_model.get_node_from_name("glycogen", compartment = "hepatocyte"),
    abm.agent_model.get_node_from_name("insulin resistance", compartment = "hepatocyte"),
], normalize = True, pi = 100, steps = None)

In [ ]:
abm.show_agents(r"LiverMap.xml", abm.agent_model.get_node_from_name("tag", compartment = "hepatocyte"))